# Org Chart Overhaul

## Objective
Given a table of employees and their managers, create three new columns:

1. **Reporting Hierarchy**: chain of command from the highest-ranking manager down to the employee  
2. **Direct Reports**: number of employees who list the person as their manager  
3. **Total Reports**: total number of employees beneath the person in the hierarchy (all descendants)

## Dataset
**File:** `OfficeSpace.csv`

Expected columns (or equivalent):
- `Employee Name`
- `Manager Name` (blank/null for the top-level manager)

## Approach

### 1) Build relationships
- Create a mapping `manager_of[employee] = manager`
- Create an adjacency list `reports[manager] = [direct_reports...]`

### 2) Reporting Hierarchy
For each employee, walk upward through `manager_of` until reaching the top-level manager, then join the path:

`Top Manager > ... > Manager > Employee`

Memoization (`lru_cache`) is used so each hierarchy path is computed once.

### 3) Direct Reports
For each employee:
- `direct_reports = len(reports[employee])`

### 4) Total Reports
For each employee, recursively count all descendants:

`total_reports(employee) = (# direct reports) + sum(total_reports(child))`

Memoization ensures efficient computation.

## Submission
Compute:

`sum(out["Total Reports"])`

This is the required answer for: **“What is the sum of the Total Reports column?”**


In [1]:
import pandas as pd
import numpy as np
from functools import lru_cache

# Load
df = pd.read_csv("OfficeSpace.csv")  # or "/mnt/data/OfficeSpace.csv"
df = df.rename(columns={"Employee Name": "employee", "Manager Name": "manager"})
df["manager"] = df["manager"].where(df["manager"].notna(), None)

# Manager mapping
manager_of = dict(zip(df["employee"], df["manager"]))

# Build direct-report adjacency (manager -> [employees])
reports = {e: [] for e in df["employee"]}
for emp, mgr in manager_of.items():
    if mgr is not None:
        reports.setdefault(mgr, []).append(emp)

# Reporting hierarchy string (top manager -> ... -> employee)
@lru_cache(None)
def hierarchy(emp: str) -> str:
    mgr = manager_of.get(emp)
    if mgr is None:
        return emp
    return hierarchy(mgr) + " > " + emp

# Total reports = all descendants count
@lru_cache(None)
def total_reports(emp: str) -> int:
    kids = reports.get(emp, [])
    return len(kids) + sum(total_reports(k) for k in kids)

# Assemble output
out = df.copy()
out["Reporting Hierarchy"] = out["employee"].map(hierarchy)
out["Direct Reports"] = out["employee"].map(lambda e: len(reports.get(e, [])))
out["Total Reports"] = out["employee"].map(total_reports)

# Final table (one row per employee)
out = out[["employee", "manager", "Reporting Hierarchy", "Direct Reports", "Total Reports"]] \
         .rename(columns={"employee": "Employee", "manager": "Manager"})

print(out.to_string(index=False))

# Submission: sum of Total Reports
sum_total_reports = int(out["Total Reports"].sum())
print("\nSum of Total Reports =", sum_total_reports)


           Employee             Manager                                                                      Reporting Hierarchy  Direct Reports  Total Reports
      Bill Lumbergh                None                                                                            Bill Lumbergh               3             24
        Bob Slydell       Bill Lumbergh                                                              Bill Lumbergh > Bob Slydell               0              0
         Bob Porter       Bill Lumbergh                                                               Bill Lumbergh > Bob Porter               0              0
   Linda M. Grayson       Bill Lumbergh                                                         Bill Lumbergh > Linda M. Grayson               4             21
       Dom Portwood    Linda M. Grayson                                          Bill Lumbergh > Linda M. Grayson > Dom Portwood               3              8
  Wendy L. Hargrove    Linda M. Grayson 